# Three Methods of Prompt Configuration

This notebook demonstrates different approaches to configuring agent prompts, from simple strings to dynamic, context-aware prompts.

## Key Concepts
- **Method 1**: String Prompt (simplest)
- **Method 2**: SystemMessage (structured)
- **Method 3**: Callable/Dynamic Prompt (advanced)

## When to Use Each Method
- **String**: Simple, static agents
- **SystemMessage**: Production chat agents
- **Callable**: Adaptive, personalized agents

In [ ]:
# Import required modules
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage
import tools

## Method 1: Simple String Prompt

The simplest approach - direct string instruction to the agent.

In [ ]:
print("=== Method 1: Simple String Prompt ===")

# Create model instance
model = ChatOllama(model="qwen3")

# Method 1: String prompt (simplest)
agent1 = create_agent(
    model,
    [tools.helper_tool],
    prompt="You are a helpful assistant. Be concise and accurate in your responses."
)

print("✓ Agent 1 created with string prompt")
print("  Characteristics:")
print("  - Direct string instruction")
print("  - Simple and readable")
print("  - Good for basic use cases")
print("  - Easy to modify")

## Method 2: SystemMessage Prompt

Using LangChain's SystemMessage class for more structured prompt handling.

In [ ]:
print("=== Method 2: SystemMessage Prompt ===")

# Method 2: SystemMessage prompt (structured)
agent2 = create_agent(
    model,
    [tools.helper_tool],
    prompt=SystemMessage(content="You are a research assistant. Always cite your sources and provide detailed explanations.")
)

print("✓ Agent 2 created with SystemMessage prompt")
print("  Characteristics:")
print("  - Structured message object")
print("  - Better integration with chat models")
print("  - More explicit about message type")
print("  - Professional for production use")

## Method 3: Dynamic Callable Prompt

A function that generates prompts based on state - the most flexible approach.

In [ ]:
print("=== Method 3: Dynamic Callable Prompt ===")

# Method 3: Callable/Dynamic prompt (most flexible)
def dynamic_prompt(state):
    """Generate prompt based on user type and context."""
    user_type = state.get("user_type", "standard")
    
    if user_type == "expert":
        system_msg = SystemMessage(content="Provide detailed technical responses with code examples and advanced concepts.")
    elif user_type == "beginner":
        system_msg = SystemMessage(content="Provide simple, clear explanations suitable for beginners. Use analogies and avoid jargon.")
    else:  # standard
        system_msg = SystemMessage(content="Provide balanced explanations that are informative but accessible.")
    
    return [system_msg] + state["messages"]

agent3 = create_agent(model, [tools.helper_tool], prompt=dynamic_prompt)

print("✓ Agent 3 created with dynamic callable prompt")
print("  Characteristics:")
print("  - Adapts to user context")
print("  - Personalized responses")
print("  - Most flexible approach")
print("  - Perfect for adaptive systems")

## Testing All Three Agents

Let's test all agents with the same question to see their different behaviors:

In [ ]:
# Test question for all agents
test_question = "Help me understand artificial intelligence"

print(f"Testing all agents with: '{test_question}'")
print("=" * 70)

# Test Agent 1 (String prompt)
print("\n=== Agent 1 Response (String Prompt) ===")
print("Expected: Concise and accurate response")

result1 = agent1.invoke({"messages": test_question})
print(f"Response: {result1['messages'][-1].content}")

# Test Agent 2 (SystemMessage prompt)
print("\n=== Agent 2 Response (SystemMessage Prompt) ===")
print("Expected: Detailed explanation with research focus")

result2 = agent2.invoke({"messages": test_question})
print(f"Response: {result2['messages'][-1].content}")


## Testing Dynamic Prompt with Different User Types

The dynamic prompt agent can adapt based on user context:

In [ ]:
print("=== Agent 3 Responses (Dynamic Prompt) ===")

# Test with expert mode
print("\n--- Expert Mode ---")
print("Expected: Technical response with advanced concepts")

result3_expert = agent3.invoke({"messages": test_question,"user_type": "expert"})
print(f"Response: {result3_expert['messages'][-1].content}")

# Test with beginner mode
print("\n--- Beginner Mode ---")
print("Expected: Simple explanation with analogies")

result3_beginner = agent3.invoke({"messages": test_question,"user_type": "beginner"})
print(f"Response: {result3_beginner['messages'][-1].content}")

# Test with standard mode (default)
print("\n--- Standard Mode ---")
print("Expected: Balanced explanation")

result3_standard = agent3.invoke({"messages": test_question,"user_type": "standard"})
print(f"Response: {result3_standard['messages'][-1].content}")

print("\n💡 Notice how each agent responds differently based on its prompt configuration")